In [1]:
import io
import json
import logging
import subprocess
import sys
from bs4 import BeautifulSoup
import warnings
import re
from contextlib import suppress
from datetime import timedelta
from functools import reduce
from logging import StreamHandler
from os import chdir, getcwd, makedirs
from os.path import dirname, exists, join
from subprocess import CalledProcessError
from typing import List, Union

import humanize
import numpy as np
import pandas as pd
import pandas_ta as ta
import requests
from google.cloud import bigquery
from moonshot.strategies.base import Moonshot
from numpy import nan
from pandas.testing import assert_frame_equal, assert_series_equal

if exists('/src'):
    WORK_DIR = '/src'
else:
    base_dir = subprocess.check_output(['git', 'rev-parse', '--show-toplevel']).strip().decode()
    WORK_DIR = join(base_dir, 'src')
    sys.path.extend([base_dir,
                     join(base_dir, 'notebook'),
                     join(base_dir, 'notebook/*')])

chdir(WORK_DIR)

warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:,.4f}'.format

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logger.addHandler(StreamHandler())

In [2]:
from src.news.provider import NewsProvider
from src.news.trading_view import TradingView
from src.models.news_model import News, Paging, PagingMetadata

GCLOUD_PROJECT = 'trading-strategy-develop'
REGION = 'asia-southeast1'

%set_env GCLOUD_PROJECT = $GCLOUD_PROJECT
%set_env REGION = $REGION

env: GCLOUD_PROJECT=trading-strategy-develop
env: REGION=asia-southeast1


In [3]:
provider: Union[NewsProvider, TradingView] = TradingView()

In [4]:
paging = provider.list(symbol='SP:SPX')

In [5]:
paging = provider.crawl(symbol='SP:SPX')

In [6]:
paging.data[100].text

'News\n/\nMT Newswires\n/\nUS Equity Investors to Watch Out for Nonfarm Payrolls This Week A…\nUS Equity Investors to Watch Out for Nonfarm Payrolls This Week Amid Expectations for Fed Rate Cut\nSep 3, 2024\n10:13 UTC\nIXIC\nSPX\nDJI\nUS equity investors will focus on nonfarm payrolls during this holiday-shortened week, as they expect the data will keep the Federal Reserve on path to begin easing policy in September, the worst month for stocks typically. \n* Having slowed sharply in July to 114,000 in part due to the impact of Hurricane Beryl on data collection, Daiwa America forecasts a "slight pickup" in nonfarm payrolls in August by around 170,000, according to a note Monday. The average increase in H1 was 218,000. This data could leave the unemployment rate unchanged at 4.3%, the joint-highest since October 2021, "although the risks are skewed to a slight decline from July."\n* Growth in average hourly earnings in the nonfarm data due Friday could remain close to the average of the

In [7]:
data = provider._detail('tag:reuters.com,2024:newsml_L1N3KL0OT')

In [8]:
html = provider.detail('https://www.tradingview.com/news/reuters.com,2024:newsml_L1N3KL0OT:0-stocks-slide-led-by-fall-in-nasdaq/', return_html=True)

In [9]:
html

'<article class="wrapper-pIO_GYwT">\n <div class="breadcrumbs-pIO_GYwT">\n  <nav aria-label="Breadcrumbs" class="breadcrumbs-container-aI9V8b_F">\n   <ul class="breadcrumbs-list-aI9V8b_F">\n    <li class="breadcrumbs-listitem-aI9V8b_F">\n     <a class="breadcrumb-cZAS4vtj text-button-H6_2ZGVv link-H6_2ZGVv text-button-light-gray-small-H6_2ZGVv typography-regular14px-H6_2ZGVv" href="/news/">\n      <span class="background-H6_2ZGVv states-without-bg-H6_2ZGVv">\n      </span>\n      <span class="content-H6_2ZGVv">\n       <span class="breadcrumb-content-cZAS4vtj">\n        News\n       </span>\n      </span>\n     </a>\n    </li>\n    <li class="breadcrumbs-listitem-aI9V8b_F">\n     <span aria-hidden="true" class="divider-cZAS4vtj divider-small-cZAS4vtj">\n      /\n     </span>\n     <a class="breadcrumb-cZAS4vtj text-button-H6_2ZGVv link-H6_2ZGVv text-button-light-gray-small-H6_2ZGVv typography-regular14px-H6_2ZGVv" href="/news/providers/reuters">\n      <span class="background-H6_2ZGVv 

In [11]:
with open('test/fixtures/news/news_headlines_v2_detail.json', 'r') as f:
    data = json.load(f)

In [84]:
with open('test/fixtures/news/news_headlines_v2_list.json', 'r') as f:
    data = json.load(f)

In [88]:
ids = [i['id'] for i in data.get('items')]

In [90]:
url = 'https://news-headlines.tradingview.com/v2/story'

headers = {}

params = {
    'id': 'tag:reuters.com,2024:newsml_L1N3KM01T:0',
    'lang': 'en',
}

response = requests.request('GET', url, headers=headers, params=params)

print(response.text)

{"shortDescription":"The following table shows rates for Asian currencies against the dollar at 0203 GMT.","astDescription":{"type":"root","children":[{"type":"p","children":["The following table shows rates for Asian currencies against the dollar at 0203 GMT."]},{"type":"table","children":[{"type":"table-body","children":[{"type":"table-row","children":[{"type":"table-data-cell","params":{"colspan":2},"children":[{"type":"p","children":["CURRENCIES VS U.S. DOLLAR"]}]},{"type":"table-data-cell","children":[]},{"type":"table-data-cell","children":[]},{"type":"table-data-cell","params":{"hidden":true}}]},{"type":"table-row","children":[{"type":"table-data-cell","params":{"colspan":2}},{"type":"table-data-cell","children":[]},{"type":"table-data-cell","children":[]},{"type":"table-data-cell","params":{"hidden":true}}]},{"type":"table-row","children":[{"type":"table-data-cell","children":[{"type":"p","children":["Currency"]}]},{"type":"table-data-cell","children":[{"type":"p","children":["

In [91]:
import requests

news_ls = []

for i in ids:
    url = 'https://news-headlines.tradingview.com/v2/story'
    headers = {}
    params = {
        'id': 'tag:reuters.com,2024:newsml_L1N3KM01T:0',
        'lang': 'en',
    }
    resp = requests.request('GET', url, headers=headers, params=params)

    news_ls.append(json.loads(resp.text))

In [95]:
json.dump(news_ls, open('abc.json', 'w'))

In [12]:
# Custom mapping from JSON types to HTML tags
type_to_tag = {
    'root': 'div',           # Use a 'div' or a different container element
    'p': 'p',
    'table': 'table',
    'table-body': 'tbody',
    'table-row': 'tr',
    'table-data-cell': 'td',
    'text': None,            # Handle text nodes separately
}


def json_to_html_with_custom_mapping(json_data):
    '''
    Function to convert JSON to HTML using custom mapping
    '''
    def convert_node(node):
        if isinstance(node, str):
            return node

        # Get the tag based on the type
        # Default to 'div' if type is unknown
        tag = type_to_tag.get(node['type'], 'html')

        # Handle text nodes (no wrapping tag)
        if node['type'] == 'text':
            return node.get('text', '')

        # Start the HTML tag
        if tag:
            html = f'<{tag}>'
        else:
            html = ''

        # Process children nodes recursively
        if 'children' in node:
            for child in node['children']:
                html += convert_node(child)

        # Close the HTML tag if necessary
        if tag:
            html += f'</{tag}>'

        return html

    return convert_node(json_data)

In [13]:
html = json_to_html_with_custom_mapping(data.get('astDescription'))

In [14]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'html.parser')
txt = soup.get_text(separator='\n')

In [17]:
soup.prettify()

'<div>\n <p>\n  The following table shows rates for Asian currencies against the dollar at 0203 GMT.\n </p>\n <table>\n  <tbody>\n   <tr>\n    <td>\n     <p>\n      CURRENCIES VS U.S. DOLLAR\n     </p>\n    </td>\n    <td>\n    </td>\n    <td>\n    </td>\n    <td>\n    </td>\n   </tr>\n   <tr>\n    <td>\n    </td>\n    <td>\n    </td>\n    <td>\n    </td>\n    <td>\n    </td>\n   </tr>\n   <tr>\n    <td>\n     <p>\n      Currency\n     </p>\n    </td>\n    <td>\n     <p>\n      Latest bid\n     </p>\n    </td>\n    <td>\n     <p>\n      Previous day\n     </p>\n    </td>\n    <td>\n     <p>\n      Pct Move\n     </p>\n    </td>\n   </tr>\n   <tr>\n    <td>\n     <p>\n      Japan yen\n     </p>\n    </td>\n    <td>\n     <p>\n      145.390\n     </p>\n    </td>\n    <td>\n     <p>\n      145.47\n     </p>\n    </td>\n    <td>\n     <p>\n      +0.06\n     </p>\n    </td>\n   </tr>\n   <tr>\n    <td>\n     <p>\n      Sing dlr\n     </p>\n    </td>\n    <td>\n     <p>\n      1.307\n     </

In [15]:
txt

'The following table shows rates for Asian currencies against the dollar at 0203 GMT.\nCURRENCIES VS U.S. DOLLAR\nCurrency\nLatest bid\nPrevious day\nPct Move\nJapan yen\n145.390\n145.47\n+0.06\nSing dlr\n1.307\n1.3071\n-0.02\nTaiwan dlr\n32.141\n32.062\n-0.25\nKorean won\n1341.600\n1345.4\n+0.28\nBaht\n34.275\n34.26\n-0.04\nPeso\n56.600\n56.5\n-0.18\nRupiah\n15530.000\n15520\n-0.06\nRupee\n83.968\n83.9675\n0.00\nRinggit\n4.350\n4.365\n+0.34\nYuan\n7.116\n7.1204\n+0.06\nChange so far in 2024\nCurrency\nLatest bid\nEnd 2023\nPct Move\nJapan yen\n145.390\n141.060\n-2.98\nSing dlr\n1.307\n1.319\n+0.90\nTaiwan dlr\n32.141\n30.735\n-4.37\nKorean won\n1341.600\n1288.000\n-4.00\nBaht\n34.275\n34.165\n-0.32\nPeso\n56.600\n55.388\n-2.14\nRupiah\n15530.000\n15395.000\n-0.87\nRupee\n83.968\n83.208\n-0.91\nRinggit\n4.350\n4.590\n+5.52\nYuan\n7.116\n7.098\n-0.25'

In [37]:
html

'The following table shows rates for Asian currencies against the dollar at 0203 GMT.CURRENCIES VS U.S. DOLLARtable-data-celltable-data-celltable-data-cellCurrencyLatest bidPrevious dayPct MoveJapan yen145.390145.47+0.06Sing dlr1.3071.3071-0.02Taiwan dlr32.14132.062-0.25Korean won1341.6001345.4+0.28Baht34.27534.26-0.04Peso56.60056.5-0.18Rupiah15530.00015520-0.06Rupee83.96883.96750.00Ringgit4.3504.365+0.34Yuan7.1167.1204+0.06Change so far in 2024table-data-cellCurrencyLatest bidEnd 2023Pct MoveJapan yen145.390141.060-2.98Sing dlr1.3071.319+0.90Taiwan dlr32.14130.735-4.37Korean won1341.6001288.000-4.00Baht34.27534.165-0.32Peso56.60055.388-2.14Rupiah15530.00015395.000-0.87Rupee83.96883.208-0.91Ringgit4.3504.590+5.52Yuan7.1167.098-0.25'

In [4]:
# symbols = [
#     'SP:S5COND',
# ]

# fields = [
#     'description'
#     'price',
#     'change_5d',
#     'change_5d_pct',
#     'low_5d',
#     'high_5d',
# ]

# results = provider.quotes(symbols=symbols, fields=fields)

In [12]:
symbols = [
    'NASDAQ:TSLA',
]

ohlcv = provider.ohclv(symbols=symbols,
                       freq='1D',
                       total_candles=252*500000).reset_index()

In [13]:
ohlcv

,Date,Symbol,Open,High,Low,Close,Volume
0,2010-06-30 13:30:00+00:00,NASDAQ:TSLA,1.71933100,2.02794500,1.55333200,1.58866500,257916256
1,2010-07-01 13:30:00+00:00,NASDAQ:TSLA,1.66666500,1.72799800,1.35133200,1.46399900,123448068
2,2010-07-02 13:30:00+00:00,NASDAQ:TSLA,1.53333200,1.53999900,1.24733300,1.27999900,77127212
3,2010-07-06 13:30:00+00:00,NASDAQ:TSLA,1.33333200,1.33333200,1.05533200,1.07399900,103189557
4,2010-07-07 13:30:00+00:00,NASDAQ:TSLA,1.09333200,1.10866600,0.99866600,1.05333200,103873812
...,...,...,...,...,...,...,...
3554,2024-08-14 13:30:00+00:00,NASDAQ:TSLA,207.39000000,208.44000000,198.75000000,201.38000000,70250014
3555,2024-08-15 13:30:00+00:00,NASDAQ:TSLA,205.02000000,215.88000000,204.82000000,214.14000000,89848530
3556,2024-08-16 13:30:00+00:00,NASDAQ:TSLA,211.15000000,219.80000000,210.80000000,216.12000000,88765122
3557,2024-08-19 13:30:00+00:00,NASDAQ:TSLA,217.07000000,222.98000000,214.09000000,222.72000000,76435222


In [14]:
ohlcv

,Date,Symbol,Open,High,Low,Close,Volume
0,2010-06-30 13:30:00+00:00,NASDAQ:TSLA,1.71933100,2.02794500,1.55333200,1.58866500,257916256
1,2010-07-01 13:30:00+00:00,NASDAQ:TSLA,1.66666500,1.72799800,1.35133200,1.46399900,123448068
2,2010-07-02 13:30:00+00:00,NASDAQ:TSLA,1.53333200,1.53999900,1.24733300,1.27999900,77127212
3,2010-07-06 13:30:00+00:00,NASDAQ:TSLA,1.33333200,1.33333200,1.05533200,1.07399900,103189557
4,2010-07-07 13:30:00+00:00,NASDAQ:TSLA,1.09333200,1.10866600,0.99866600,1.05333200,103873812
...,...,...,...,...,...,...,...
3554,2024-08-14 13:30:00+00:00,NASDAQ:TSLA,207.39000000,208.44000000,198.75000000,201.38000000,70250014
3555,2024-08-15 13:30:00+00:00,NASDAQ:TSLA,205.02000000,215.88000000,204.82000000,214.14000000,89848530
3556,2024-08-16 13:30:00+00:00,NASDAQ:TSLA,211.15000000,219.80000000,210.80000000,216.12000000,88765122
3557,2024-08-19 13:30:00+00:00,NASDAQ:TSLA,217.07000000,222.98000000,214.09000000,222.72000000,76435222


In [15]:
provider.calc_perf(ohlcv, '24h')

{'NASDAQ:TSLA': {'close': 221.1,
  'change_24h': -1.6200000000000045,
  'change_24h_pct': -0.007273706896551745,
  'low_24h': 219.56,
  'high_24h': 228.22}}

In [7]:
provider.calc_perf(ohlcv, '5D')

{'NASDAQ:TSLA': {'close': 221.08,
  'change_5d': -0.7800000000000011,
  'change_5d_pct': -0.0035157306409447447,
  'low_5d': 221.0,
  'high_5d': 221.93}}

In [9]:
provider.calc_perf(ohlcv, '1M')

{'NASDAQ:TSLA': {'close': 221.08,
  'change_1m': -0.39999999999997726,
  'change_1m_pct': -0.0018060321473721208,
  'low_1m': 221.0,
  'high_1m': 222.47}}

In [8]:
provider.calc_perf(ohlcv, 'MTD')

{'SP:S5COND': {'close': 1463.72,
  'change_mtd': 40.22000000000003,
  'change_mtd_pct': 0.028254302774850738,
  'low_mtd': 1411.67,
  'high_mtd': 1473.95}}

In [9]:
provider.calc_perf(ohlcv, 'YTD')

{'SP:S5COND': {'close': 1463.72,
  'change_ytd': 45.6400000000001,
  'change_ytd_pct': 0.03218436195419166,
  'low_ytd': 1360.78,
  'high_ytd': 1493.28}}